In [2]:
%load_ext autoreload

In [3]:
%autoreload 

from connector import DatabaseRedshift
import pandas as pd
import numpy as np
import os

**Test small dataframe inserts**

In [5]:
schema = 'sandbox_ana'
table = 'rory_test'

In [6]:

df_ = pd.DataFrame({'a':[1, 2.0, 3, 4, 5], 
                  'b': [1.0, 2.0, 3.0, np.nan, 6.0], 
                  'c':[1.11111111, 1.0, 3.333333, np.nan, 4.444], 
                  'd':[10**7, np.nan, np.nan, np.nan, 40**7], 
                  'e':[100000.10202020101, 1000000000.20920922, 100000000000.2902902902, np.nan, 100000.089309],
                  'f':['hello', np.nan, 'hello', np.nan, 'goodbye'],
                  'g':[True, False, False, np.nan, np.nan],
                  'h':[np.nan, np.nan, np.nan, np.nan, np.nan],
                   'i':['hello', 'hello', 'hello', 'hello', np.nan]})
print(df_)

     a    b         c             d             e        f      g   h      i
0  1.0  1.0  1.111111  1.000000e+07  1.000001e+05    hello   True NaN  hello
1  2.0  2.0  1.000000           NaN  1.000000e+09      NaN  False NaN  hello
2  3.0  3.0  3.333333           NaN  1.000000e+11    hello  False NaN  hello
3  4.0  NaN       NaN           NaN           NaN      NaN    NaN NaN  hello
4  5.0  6.0  4.444000  1.638400e+11  1.000001e+05  goodbye    NaN NaN    NaN


In [7]:
with DatabaseRedshift() as db:
    db.insert(df=df_, schema=schema, table=table, mode='replace')

Data written to sandbox_ana.rory_test


In [8]:
with DatabaseRedshift() as db:
    db.insert(df=df_, schema=schema, table=table, mode='append')

Data written to sandbox_ana.rory_test


In [9]:
df = pd.DataFrame({'a':["hello's", "goodbye's"]})
with DatabaseRedshift() as db:
    db.insert(df=df, schema=schema, table=table, mode='replace')

Data written to sandbox_ana.rory_test


In [10]:
df = pd.DataFrame({'a':[np.nan, np.nan, np.nan]})
with DatabaseRedshift() as db:
    db.insert(df=df, schema=schema, table=table, mode='replace')

Data written to sandbox_ana.rory_test


In [16]:
frames = []
for i in range(2000):
    frames.append(df_)
df_1 = pd.concat(frames)

In [18]:
with DatabaseRedshift() as db:
    db.insert(df=df_, schema=schema, table=table, mode='replace')

Data written to sandbox_ana.rory_test


In [21]:
with DatabaseRedshift() as db:
    db.insert(df=df_1, schema=schema, table=table, mode='append', chunksize=2000)

Data written to sandbox_ana.rory_test


**Writing large dataframes**

In [11]:
df = pd.concat([df_, df_])
df.reset_index(inplace=True, drop=True)
df = df.reindex(np.arange(10000))

In [13]:
with DatabaseRedshift() as db:
    db.insert(df=df, schema='sandbox_ana', table='rory_datatypes_test', mode='replace')

Table sandbox_ana.rory_datatypes_test created successfully
No name passed to `s3_csv_name` defaulting to 20190916_165345_df.csv
Saved dwh/20190916_165345_df.csv to bse-analytics-dev.bseint.io
Data written to sandbox_ana.rory_datatypes_test


In [14]:
df = pd.DataFrame({'a':[1, 2.0, 3, 4, 5], 
                  'b': [1.0, 2.0, 3.0, np.nan, 6.0], 
                  'c':[1.11111111, np.nan, 3.333333, np.nan, 4.444], 
                  'd':[10**7, np.nan, np.nan, np.nan, 40**7], 
                  'e':[100000.10202020101, 1000000000.20920922, 100000000000.2902902902, np.nan, 100000.089309],
                  'f':['hello', np.nan, 'hello', np.nan, 'goodbye'],
                  'g':[True, False, False, np.nan, np.nan],
                  'h':[np.nan, np.nan, np.nan, np.nan, np.nan]})
print(df)

     a    b         c             d             e        f      g   h
0  1.0  1.0  1.111111  1.000000e+07  1.000001e+05    hello   True NaN
1  2.0  2.0       NaN           NaN  1.000000e+09      NaN  False NaN
2  3.0  3.0  3.333333           NaN  1.000000e+11    hello  False NaN
3  4.0  NaN       NaN           NaN           NaN      NaN    NaN NaN
4  5.0  6.0  4.444000  1.638400e+11  1.000001e+05  goodbye    NaN NaN


In [15]:
with DatabaseRedshift() as db:
    db.insert(df=df.head(1), schema='sandbox_ana', table='rory_datatypes_test', mode='replace')

Data written to sandbox_ana.rory_datatypes_test


In [16]:
with DatabaseRedshift() as db:
    db.insert(df=df.reindex(np.arange(5005)).ffill(), schema='sandbox_ana', table='rory_datatypes_test', mode='replace')

Table sandbox_ana.rory_datatypes_test created successfully
No name passed to `s3_csv_name` defaulting to 20190916_165352_df.csv
Saved dwh/20190916_165352_df.csv to bse-analytics-dev.bseint.io
Data written to sandbox_ana.rory_datatypes_test


In [17]:
with DatabaseRedshift() as db:
    db.insert(df=df, schema='sandbox_ana', table='rory_datatypes_test', mode='replace')

Data written to sandbox_ana.rory_datatypes_test


In [18]:
with DatabaseRedshift() as db:
    db.insert(df=df, schema='sandbox_ana', table='rory_datatypes_test', mode='fail')

ValueError: Table 'rory_datatypes_test' already exists.

In [19]:
with DatabaseRedshift() as db:
    db.insert(df=df, schema='sandbox_ana', table='rory_datatypes_test', mode='append')

Data written to sandbox_ana.rory_datatypes_test


In [20]:
with DatabaseRedshift() as db:
    df_r = db.fetch("select * from sandbox_ana.rory_datatypes_test")

In [21]:
print(pd.concat([df_r.dtypes, df.dtypes], axis=1))
# We expect all floats as we have nan's in our data -> pandas automatically converts this to float.

         0        1
a  float64  float64
b  float64  float64
c  float64  float64
d  float64  float64
e  float64  float64
f   object   object
g   object   object
h  float64  float64


In [22]:
# Test dtype conversions
with DatabaseRedshift() as db:
    df = db.fetch("select * from sandbox_ana.yanan_bba1597_journey limit 100")

In [23]:
print(df.dtypes)

referenceorder         object
sku                     int64
itemretailpriceeur    float64
itemsaleseur          float64
dtype: object


In [24]:
# Test append mode when table does not exist
with DatabaseRedshift() as db:
    db.insert(df=df, schema='sandbox_ana', table='blablablalkalalal', mode='append')

Data written to sandbox_ana.blablablalkalalal


**Testing large dataframes**

In [25]:
df = df.reindex(np.arange(1, 50000)).ffill()

In [26]:
len(df)

49999

In [27]:
with DatabaseRedshift() as db:
    db.insert(
        df=df, schema='sandbox_ana', table='rory_datatypes_test', s3_csv_name='test.csv', mode='replace'
    )

Table sandbox_ana.rory_datatypes_test created successfully
Saved dwh/test.csv to bse-analytics-dev.bseint.io
Data written to sandbox_ana.rory_datatypes_test


In [28]:
with DatabaseRedshift() as db:
    db.insert(
        df=df, schema='sandbox_ana', table='rory_datatypes_test', s3_csv_name='test.csv', mode='fail'
    )

Exception: Table sandbox_ana.rory_datatypes_test exists!

In [29]:
with DatabaseRedshift() as db:
    db.insert(
        df=df, schema='sandbox_ana', table='rory_datatypes_test', s3_csv_name='test.csv', mode='append'
    )

Data written to sandbox_ana.rory_datatypes_test
